In [1]:
import cv2
import dlib
import numpy as np
import scipy as sp
from scipy.sparse import dok_matrix

In [15]:
l = list()

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks_GTX.dat")
cap = cv2.VideoCapture("test.mov")

fps = cap.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count/fps

while cap.isOpened():
    # Capture the image from the webcam
    ret, image = cap.read()
    if image is None:
        break
    height, width = image.shape[:2]
    if height < width:
        new_height = int(height / width * 480)
        new_width = 480
    else:
        new_width = int(width / height * 480)
        new_height = 480
    image = cv2.resize(image, (new_width, new_height))
    # Convert the image color to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Detect the face
    rects = detector(gray, 1)
    # Detect landmarks for each face
    for rect in rects:
        # Get the landmark points
        shape = predictor(gray, rect)
	# Convert it to the NumPy Array
        shape_np = np.zeros((68, 2), dtype="int")
        for i in range(0, 68):
            shape_np[i] = (shape.part(i).y, shape.part(i).x)
        shape = shape_np
        l.append(shape)
cap.release()



In [17]:
cpt = 0
for frame in l:
    res = np.ones((new_height, new_width)) * 255
    for k, v in frame:
        res[k,v] = 0
    cv2.imwrite('frames/frame{:04d}.png'.format(cpt), res)
    cpt += 1


In [20]:
import glob
 
img_array = []
for filename in glob.glob('frames/*.png'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('out.mp4',cv2.VideoWriter_fourcc(*'mp4v'), 30, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()